### Figure Friday 2024 - W50
In this week’s Figure-Friday we’ll look at Commodity Markets and their monthly prices, provided by the[World Bank Group](https://github.com/plotly/Figure-Friday/tree/main/2024/week-50). In case you’re interested in more info such as monthly indices for particular sectors (energy, agriculture, etc.) or a data description, simply download this [Worldbank excel sheet](https://thedocs.worldbank.org/en/doc/5d903e848db1d1b83e0ec8f744e55570-0350012021/related/CMO-Historical-Data-Monthly.xlsx) and view the other 3 sheets/tabs: Monthly Indices, Description, and Index Weights.</br>

One initial approach was trying to create a hierarchical chart, such as a Sunburst due to the quality of the data-groupers, but in my case, it didn’t get me ‘home and dry’. So, I changed my perspective and started to look for correlations. Obviously, there are lots of them in this kind of data. The first line-chart plotted represents just prices in a time-line-chart but grouped by sectors in a clean plot and it is followed by the correlation matrix from this particular group. The correlation matrix is based on [Spearman correlation coefficient.](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)

In [2]:
"""Just importing"""
from dash import Dash, html, Output, Input, State, callback, dcc, no_update, ctx
import dash_mantine_components as dmc
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from dash_bootstrap_templates import load_figure_template

# print(pio.templates.default)
pio.templates.default = 'plotly_white'

df = pd.read_csv("https://raw.githubusercontent.com/plotly/Figure-Friday/refs/heads/main/2024/week-50/CMO-Historical-Data-Monthly.csv")
df = df.drop([0, 1])  # drop sub headers
df.rename(columns={'Unnamed: 0': 'Time'}, inplace=True)
df['Time'] = pd.to_datetime(df['Time'], format='%YM%m')

df.set_index('Time', inplace=True)
dff = df.iloc[:,:].apply(pd.to_numeric, errors='coerce')
dff.fillna(0,inplace=True)

# Convert all columns (except 'Time') to numeric ==>> didn't work
# df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
# dff = df.set_index('Time').convert_dtypes(convert_string=False).fillna(0)


# Preparing cols to plot group of commodities
# energy_cols = dff.columns[:10]#.tolist()
# bevg_cols = dff.columns[10:17].tolist()
# oils_meals = dff.columns[17:28].tolist()
# cereals = dff.columns[28:37].tolist()
# other_food = dff.columns[37:47].tolist()
# agric_cols = dff.columns[47:56].tolist()
# fert_cols = dff.columns[56:61].tolist()
# met_cols = dff.columns[61:].tolist()

get_cols = {
    'energy_cols': ['Crude oil, average', 'Crude oil, Brent', 'Crude oil, Dubai', 'Crude oil, WTI', 'Coal, Australian',\
                     'Coal, South African **', 'Natural gas, US', 'Natural gas, Europe', 'Liquefied natural gas, Japan', 'Natural gas index'],
    'bevg_cols': ['Cocoa', 'Coffee, Arabica', 'Coffee, Robusta', 'Tea, avg 3 auctions', 'Tea, Colombo', 'Tea, Kolkata', 'Tea, Mombasa'],
    'oils_meals': ['Coconut oil', 'Groundnuts', 'Fish meal', 'Groundnut oil **', 'Palm oil', 'Palm kernel oil', 'Soybeans', 'Soybean oil',\
                    'Soybean meal', 'Rapeseed oil', 'Sunflower oil'],
    'cereals': ['Barley', 'Maize', 'Sorghum', 'Rice, Thai 5% ', 'Rice, Thai 25% ', 'Rice, Thai A.1', 'Rice, Viet Namese 5%', 'Wheat, US SRW', 'Wheat, US HRW'],
    'other_food': ['Banana, Europe', 'Banana, US', 'Orange', 'Beef **', 'Chicken **', 'Lamb **', 'Shrimps, Mexican', 'Sugar, EU', 'Sugar, US', 'Sugar, world'],
    'agric_cols': ['Tobacco, US import u.v.', 'Logs, Cameroon', 'Logs, Malaysian', 'Sawnwood, Cameroon', 'Sawnwood, Malaysian', 'Plywood',\
                    'Cotton, A Index', 'Rubber, TSR20 **', 'Rubber, RSS3'],
    'fert_cols': ['Phosphate rock', 'DAP', 'TSP', 'Urea ', 'Potassium chloride **'],
    'met_cols': ['Aluminum', 'Iron ore, cfr spot', 'Copper', 'Lead', 'Tin', 'Nickel', 'Zinc', 'Gold', 'Platinum', 'Silver']
}

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

range_slider = dcc.RangeSlider(
        id='year_slider',
        min = dff.index.min().year,
        max = dff.index.max().year,
        step = 2,
        tooltip={
                    "always_visible": True,
                    "template": "{value}",
                    "placement":'bottom',
                },
        marks={year: str(year) for year in range(dff.index.min().year, dff.index.max().year, 5)},
        value=[1990, 2024]
)

radio_group = dmc.RadioGroup(
    id= 'radio_groups',
    label='Select group to plot',
    value=[],
    size='sm',
    # mb=5,
    children= dmc.Group([
        dmc.Radio(label='Energy', value='energy_cols'),
        dmc.Radio(label='Beverages', value='bevg_cols'),
        dmc.Radio(label='Oils & Meals', value='oils_meals'),
        dmc.Radio(label='Cereals', value='cereals'),
        dmc.Radio(label='Other Food', value='other_food'),
        dmc.Radio(label='Agriculture', value='agric_cols'),
        dmc.Radio(label='Fertilizer', value='fert_cols'),
        dmc.Radio(label='Metals', value='met_cols'),
    ], className='m-2'), className='border ps-1 my-1',
)

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H3("Commodity Markets and their monthly prices",
                        className="text-start text-primary my-3"),
                width=11)
    ], justify='around'), #align='center',
    dbc.Row([
        dbc.Col([
            html.Label('Select year range:', className='text-start text-primary'),
            range_slider
        ], width=11)
    ], justify='around'),
    dbc.Row([
        dbc.Col(radio_group, width=11)
    ], align='center', justify='around'),
    dbc.Row([
        dbc.Col(dcc.Graph(id="line_chart", figure={}, className='shadow rounded'), width=11),
    ], justify='around'), #align='center',
    dbc.Row([
        dbc.Col([
            html.Label('Correlation Matrix for selected group plotted', className='text-primary my-3'),
            dcc.Graph(id='corr_mat', figure={}, className='shadow rounded')], 
            width=11)
    ], justify='around')
], fluid=True)

@callback(
    Output('line_chart', 'figure'),
    Input('radio_groups', 'value'),
    Input('year_slider', 'value'),
    prevent_initial_call=True
)
def update_line_chart(value, slider_value):
    cols = get_cols[value]
    new_df = dff[cols]
    # print(cols)
    # print(new_df.info())

    if ctx.triggered_id == 'radio_groups':
        fig = px.line(new_df, x=new_df.index, y=cols,
                    labels={'variable':'', 'Time':'', 'value':''})
        fig.update_layout(hovermode='x', legend_orientation='h')
        # https://plotly.com/python/hover-text-and-formatting/#advanced-hover-template  >> to check hover data

        return fig
    
    else: 
        year_st = str(slider_value[0])
        year_end = str(slider_value[1])
        new_dff = new_df.loc[year_st:year_end]
        fig = px.line(new_dff, x=new_dff.index, y=cols,
            labels={'variable':'', 'Time':'', 'value':''})
        fig.update_layout(hovermode='x', legend_orientation='h')

        return fig
    # else: return no_update

@callback(
    Output('corr_mat', 'figure'),
    Input('radio_groups', 'value'),
    prevent_initial_call=True
)
def update_corr_mat(value):
    cols = get_cols[value]
    new_df = dff[cols]

#     # https://numpy.org/doc/stable/reference/generated/numpy.tril.html#numpy.tril
#     # np.tril(np.ones(corr_df.shape)).astype(bool)[0:5,0:5]
    corr_mat = new_df.corr(method='spearman')
    corr_mat_trim = corr_mat.where(np.tril(np.ones(corr_mat.shape), k=-1).astype(bool))
    fig2 = px.imshow(corr_mat_trim, aspect='auto',
                    color_continuous_scale=[(0, "yellow"), (0.6, "orange"), (1, "brown")],
                    template='simple_white',
                    height=600)#'YlGnBu_r', # color_continuous_midpoint=0.5)


    return fig2

if __name__ == '__main__':
    app.run(debug=True, port= 9090, jupyter_mode='external')

Dash app running on http://127.0.0.1:9090/
